# DLC Live PyTorch Demo

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dlclive import DLCLive, Processor
from dlclive.display import Display
import cv2
import numpy as np
from pathlib import Path
import time

Loading DLC 3.0.0rc4...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


/home/dikra/miniconda3/envs/dlc-live/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Snapshot to ONNX model 

In [6]:
# In case you do not have a .onnx model exported, use this cell to export your DLC3.0 snapshot

from deeplabcut.pose_estimation_pytorch.config import read_config_as_dict
from deeplabcut.pose_estimation_pytorch.models import PoseModel
import torch
import onnxruntime as ort

device = "cuda" if torch.cuda.is_available() else "cpu"

root = Path("/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy/")
model_cfg = read_config_as_dict(root / "pytorch_config.yaml")
weights_path = root / "snapshot-200.pt"

model = PoseModel.build(model_cfg["model"])
weights = torch.load(weights_path, map_location=device)
model.load_state_dict(weights["model"])

dummy_input = torch.zeros((1, 3, 224, 224))

torch.onnx.export(
    model,
    dummy_input,
    "/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy/resnet.onnx",
    verbose=False,
    input_names=["input"],
    dynamic_axes={"input": {0: "batch_size", 2: "height", 3: "width"}},
)

/tmp/ipykernel_34268/968774635.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights = torch.load(weights_path, map_location=device)


In [34]:
# Load test frame
img = cv2.imread("/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy/img008.png")

### DLC Live with ONNX exported DLC 3.0 model

In [35]:
# Dikra
onnx_dlc_live = DLCLive(
    path="/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy",
    model_type="onnx",
    device="cuda",
    display=True,
)

# Anna
# onnx_dlc_live = DLCLive(pytorch_cfg="/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/train", processor=dlc_proc, snapshot='/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/train/snapshot-263.pt')
# onnx_dlc_live = DLCLive("/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/exported DLC model for dlc-live/DLC_dev-single-animal_resnet_50_iteration-1_shuffle-1", processor=dlc_proc)
# img = cv2.imread("/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/exported DLC model for dlc-live/img049.png")

onnx_pose = onnx_dlc_live.init_inference(frame=img)
onnx_pose

Loading the model took 0.22803378105163574 sec
ONNX inference took 1.7877936363220215 sec


{'poses': tensor([[[[3.6219e+02, 4.1237e+02, 5.0484e-01],
           [3.6271e+02, 3.9583e+02, 5.8567e-01],
           [3.5676e+02, 3.7445e+02, 7.0499e-01],
           [3.4043e+02, 3.5935e+02, 7.1168e-01],
           [3.3437e+02, 4.0497e+02, 6.5114e-01],
           [3.3772e+02, 4.0690e+02, 4.4899e-01],
           [2.0013e+02, 4.3546e+02, 5.2085e-01],
           [3.4130e+02, 4.3089e+02, 3.4881e-01],
           [2.1540e+02, 3.9427e+02, 7.8372e-01],
           [1.9975e+02, 3.7728e+02, 5.2776e-01],
           [1.8255e+02, 3.6741e+02, 6.4682e-01],
           [2.7520e+02, 3.3175e+02, 5.7621e-01],
           [3.5014e+02, 4.3089e+02, 4.1483e-01],
           [4.3459e+02, 3.1565e+02, 8.1688e-01],
           [4.2561e+02, 2.9912e+02, 8.8812e-01],
           [2.4359e+02, 5.0566e+02, 5.7648e-01],
           [3.5792e+02, 4.8275e+02, 3.0060e-01],
           [4.0234e+02, 4.1449e+02, 5.4227e-01],
           [1.7143e+02, 4.8667e+02, 4.5833e-01],
           [3.2110e+02, 5.1144e+02, 2.5105e-01],
           

![Screenshot from 2024-08-20 14-29-53.png](./docs/assets/Screenshot%20from%202024-08-20%2014-36-00.png)

In [36]:
onnx_pose = onnx_dlc_live.get_pose(frame=img)
onnx_pose

ONNX inference took 0.06539440155029297 sec


{'poses': tensor([[[[3.6219e+02, 4.1237e+02, 5.0484e-01],
           [3.6271e+02, 3.9583e+02, 5.8567e-01],
           [3.5676e+02, 3.7445e+02, 7.0499e-01],
           [3.4043e+02, 3.5935e+02, 7.1168e-01],
           [3.3437e+02, 4.0497e+02, 6.5114e-01],
           [3.3772e+02, 4.0690e+02, 4.4899e-01],
           [2.0013e+02, 4.3546e+02, 5.2085e-01],
           [3.4130e+02, 4.3089e+02, 3.4881e-01],
           [2.1540e+02, 3.9427e+02, 7.8372e-01],
           [1.9975e+02, 3.7728e+02, 5.2776e-01],
           [1.8255e+02, 3.6741e+02, 6.4682e-01],
           [2.7520e+02, 3.3175e+02, 5.7621e-01],
           [3.5014e+02, 4.3089e+02, 4.1483e-01],
           [4.3459e+02, 3.1565e+02, 8.1688e-01],
           [4.2561e+02, 2.9912e+02, 8.8812e-01],
           [2.4359e+02, 5.0566e+02, 5.7648e-01],
           [3.5792e+02, 4.8275e+02, 3.0060e-01],
           [4.0234e+02, 4.1449e+02, 5.4227e-01],
           [1.7143e+02, 4.8667e+02, 4.5833e-01],
           [3.2110e+02, 5.1144e+02, 2.5105e-01],
           

In [37]:
onnx_dlc_live.display.destroy()

### DLC Live with snaptshot of DLC 3.0 model (.pt)

In [7]:
# Dikra
pytorch_dlc_live = DLCLive(
    path="/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy",
    snapshot="snapshot-200.pt",
    device="cuda",
    model_type="pytorch",
    display=True,
)

# Anna
# pytorch_dlc_live = DLCLive(pytorch_cfg="/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/train", processor=dlc_proc, snapshot='/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/Ventral_gait_model/train/snapshot-263.pt')
# pytorch_dlc_live = DLCLive("/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/exported DLC model for dlc-live/DLC_dev-single-animal_resnet_50_iteration-1_shuffle-1", processor=dlc_proc)
# img = cv2.imread("/Users/annastuckert/Documents/DLC_AI_Residency/DLC_AI2024/DeepLabCut-live/exported DLC model for dlc-live/img049.png")

pytorch_pose = pytorch_dlc_live.init_inference(frame=img)
pytorch_pose

/home/dikra/MyHub/Code/DLC24_Hub/DLC_AI2024/DeepLabCut-live/dlclive/dlclive.py:260: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights = torch.load(model_path, map_locati

{'poses': tensor([[[[4.6099e+02, 3.0565e+02, 2.2497e-01],
           [4.6334e+02, 2.9737e+02, 5.2393e-01],
           [4.7105e+02, 2.9226e+02, 5.7827e-01],
           [4.7816e+02, 2.9004e+02, 6.5970e-01],
           [5.7700e+02, 3.4287e+02, 6.7695e-01],
           [4.4610e+02, 3.3317e+02, 6.4596e-01],
           [5.3632e+02, 4.2195e+02, 5.4373e-01],
           [4.5563e+02, 3.5570e+02, 2.5123e-01],
           [5.5438e+02, 3.6476e+02, 6.2622e-01],
           [5.3584e+02, 3.6377e+02, 6.4735e-01],
           [5.1307e+02, 3.6640e+02, 6.0587e-01],
           [5.2159e+02, 2.9548e+02, 4.5409e-01],
           [4.8214e+02, 3.8471e+02, 2.4118e-01],
           [4.8334e+02, 2.1336e+02, 7.4011e-01],
           [4.5919e+02, 2.0870e+02, 6.2153e-01],
           [6.0561e+02, 4.7224e+02, 5.8598e-01],
           [5.9742e+02, 4.2868e+02, 4.8355e-01],
           [5.7910e+02, 4.2470e+02, 2.6089e-01],
           [4.6775e+02, 4.9880e+02, 8.5475e-01],
           [4.4069e+02, 4.2392e+02, 6.7025e-01],
           

In [8]:
pytorch_dlc_live.display.destroy()

![PyTorch model inference](./docs/assets/Screenshot%20from%202024-08-20%2014-29-53.png)

### Which is faster?

In [3]:
import glob
import os

root = "/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy"
test_images = glob.glob(os.path.normpath(root + "/*.png"))


def mean_time_inference(dlc_live, images):
    times = []
    for i, img_p in enumerate(images):
        img = cv2.imread(img_p)

        if i == 0:
            start = time.time()
            dlc_live.init_inference(img)
            end = time.time()
        else:
            start = time.time()
            dlc_live.get_pose(img)
            end = time.time()
        times.append(end - start)
    print(times)

    return np.mean(times), times

In [6]:
dlc_live = DLCLive(
    path="/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy",
    device="cuda",
    model_type="onnx",
    display=True,
)

mean_time = mean_time_inference(dlc_live, test_images)
print(
    f"TOTAL Inference of ONNX model took on average {mean_time} seconds for {len(test_images)} images"
)

Loading the model took 0.5293464660644531 sec
ONNX inference took 2.8643910884857178 sec
ONNX inference took 0.06293821334838867 sec
ONNX inference took 0.06203794479370117 sec
ONNX inference took 0.06301212310791016 sec
ONNX inference took 0.06281137466430664 sec
ONNX inference took 0.06262016296386719 sec
ONNX inference took 0.06253719329833984 sec
ONNX inference took 0.06359624862670898 sec
ONNX inference took 0.06127595901489258 sec
ONNX inference took 0.0638742446899414 sec
ONNX inference took 0.06360268592834473 sec
ONNX inference took 0.06196093559265137 sec
ONNX inference took 0.06411862373352051 sec
ONNX inference took 0.062486886978149414 sec
ONNX inference took 0.06318998336791992 sec
ONNX inference took 0.06252145767211914 sec
ONNX inference took 0.06220364570617676 sec
ONNX inference took 0.06342172622680664 sec
ONNX inference took 0.06285452842712402 sec
ONNX inference took 0.061716318130493164 sec
ONNX inference took 0.0622098445892334 sec
ONNX inference took 0.061321496

In [7]:
dlc_live.display.destroy()

In [29]:
dlc_live = DLCLive(
    path="/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy",
    snapshot="snapshot-200.pt",
    device="cuda",
    model_type="pytorch",
    display=True,
)

mean_time = mean_time_inference(dlc_live, test_images)
print(
    f"Inference of PyTorch model took on average {mean_time} seconds for {len(test_images)} images"
)

/home/dikra/MyHub/Code/DLC24_Hub/DLC_AI2024/DeepLabCut-live/dlclive/dlclive.py:257: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.pose_model = PoseModel.build(self.cfg[

Loading the model took 0.2649521827697754 sec
PyTorch inference took 0.004726886749267578 sec
PyTorch inference took 0.013677835464477539 sec
PyTorch inference took 0.03649282455444336 sec
PyTorch inference took 0.032695770263671875 sec
PyTorch inference took 0.008616924285888672 sec
PyTorch inference took 0.012260675430297852 sec
PyTorch inference took 0.039740800857543945 sec
PyTorch inference took 0.016360998153686523 sec
PyTorch inference took 0.029901742935180664 sec
PyTorch inference took 0.013670206069946289 sec
PyTorch inference took 0.006944894790649414 sec
PyTorch inference took 0.015252113342285156 sec
PyTorch inference took 0.00914621353149414 sec
PyTorch inference took 0.028045177459716797 sec
PyTorch inference took 0.027249574661254883 sec
PyTorch inference took 0.028400182723999023 sec
PyTorch inference took 0.02504706382751465 sec
PyTorch inference took 0.03605461120605469 sec
PyTorch inference took 0.019296646118164062 sec
PyTorch inference took 0.016387224197387695 se

In [4]:
dlc_live = DLCLive(
    path="/media/dikra/PhD/DATA/DLC24_Data/dlc-live-dummy",
    device="tensorrt",
    model_type="onnx",
    display=True,
)

mean_time = mean_time_inference(dlc_live, test_images)
print(
    f"Inference of PyTorch model took on average {mean_time} seconds for {len(test_images)} images"
)

*************** EP Error ***************
EP Error /onnxruntime_src/onnxruntime/python/onnxruntime_pybind_state.cc:490 void onnxruntime::python::RegisterTensorRTPluginsAsCustomOps(PySessionOptions&, const onnxruntime::ProviderOptions&) Please install TensorRT libraries as mentioned in the GPU requirements page, make sure they're in the PATH or LD_LIBRARY_PATH, and that your GPU is supported.
 when using ['TensorrtExecutionProvider']
Falling back to ['CPUExecutionProvider'] and retrying.
****************************************
Loading the model took 0.4814929962158203 sec


2024-08-21 14:16:18.942517786 [E:onnxruntime:Default, provider_bridge_ort.cc:1978 TryGetProviderInfo_TensorRT] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1637 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_tensorrt.so with error: libnvinfer.so.10: cannot open shared object file: No such file or directory



ONNX inference took 0.6044352054595947 sec
ONNX inference took 0.6298971176147461 sec
ONNX inference took 0.5582480430603027 sec
ONNX inference took 0.5535740852355957 sec
ONNX inference took 0.5072672367095947 sec
ONNX inference took 0.48339295387268066 sec


KeyboardInterrupt: 

In [9]:
dlc_live.sess.get_provider_options()

{'CPUExecutionProvider': {},
 'CUDAExecutionProvider': {'sdpa_kernel': '0',
  'use_tf32': '1',
  'prefer_nhwc': '0',
  'tunable_op_max_tuning_duration_ms': '0',
  'enable_skip_layer_norm_strict_mode': '0',
  'tunable_op_tuning_enable': '0',
  'tunable_op_enable': '0',
  'use_ep_level_unified_stream': '0',
  'device_id': '0',
  'has_user_compute_stream': '0',
  'gpu_external_empty_cache': '0',
  'cudnn_conv_algo_search': 'EXHAUSTIVE',
  'cudnn_conv1d_pad_to_nc1d': '0',
  'gpu_mem_limit': '18446744073709551615',
  'gpu_external_alloc': '0',
  'gpu_external_free': '0',
  'arena_extend_strategy': 'kNextPowerOfTwo',
  'do_copy_in_default_stream': '1',
  'enable_cuda_graph': '0',
  'user_compute_stream': '0',
  'cudnn_conv_use_max_workspace': '1'}}

In [41]:
dlc_live.display.destroy()